CIFAR10 (Canadian Institute for Advanced Research) small images classification dataset- dataset of 50,000 32x32 color training images and 10,000 test images, labeled over 10 categories
| **Label Index** | **Class Name** | **Example Object(s)**            |
| --------------- | -------------- | -------------------------------- |
| 0               | Airplane       | Jet, helicopter, passenger plane |
| 1               | Automobile     | Car, SUV, sedan                  |
| 2               | Bird           | Sparrow, parrot, eagle           |
| 3               | Cat            | Domestic cat, kitten             |
| 4               | Deer           | Deer, elk                        |
| 5               | Dog            | Domestic dog, puppy              |
| 6               | Frog           | Tree frog, toad                  |
| 7               | Horse          | Horse, pony                      |
| 8               | Ship           | Boat, ocean liner, sailboat      |
| 9               | Truck          | Pickup, delivery truck           |


In [1]:
import numpy as np
import pandas as pd
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.callbacks import LearningRateScheduler

from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [2]:
# import dataset
from tensorflow.keras.datasets import cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [3]:
def data_summary(x_train,y_train,x_test,y_test):
    print('x_train.shape:',x_train.shape)  #(num_samples, height, width, channels)  channels - RGB
    print('y_train.shape:',y_train.shape)
    print('x_test.shape:',x_test.shape)
    print('y_test.shape:',y_test.shape)

In [4]:
data_summary(x_train,y_train,x_test,y_test)

x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 1)
x_test.shape: (10000, 32, 32, 3)
y_test.shape: (10000, 1)


In [5]:
x_train.shape[1:]

(32, 32, 3)

In [6]:
# Standardisation:

train_mean = np.mean(x_train, axis= (0,1,2))
train_std = np.std(x_train, axis= (0,1,2))
x_train = (x_train-train_mean)/ (train_std + 1e-7)
x_test = (x_test-train_mean)/ (train_std +1e-7)

In [7]:
print(train_mean.shape)
print(train_std.shape)

(3,)
(3,)


In [8]:
#one hot encoding of y labels:
num_classes = 10
y_train = to_categorical(y_train,num_classes)
y_test =  to_categorical(y_test,num_classes)

<!-- elu-- (Exponential Linear Unit) is an activation function
ELU(x)={x        if x>0
        α(e^x−1) if x≤0    Where α (alpha) is a hyperparameter (typically α = 1.0).
diagram:
          /
         /
        /
-------/
      /
    /
For positive values: behaves like ReLU (linear).

For negative values: instead of going flat (like ReLU), it decays smoothly toward -α.
 -->


<!-- #######old code#####
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('elu'))  
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Activation('elu'))  
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Activation('elu'))  
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Activation('elu'))  
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Activation('elu'))  
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(BatchNormalization())
model.add(Activation('elu'))  
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# DO activation after normalisation   Conv → BatchNorm → Activation → Pooling → Dropout  best pratices -->

In [9]:
# Model Building

weight_decay = 1e-4  # Regularization strength to prevent overfitting
num_classes = 10     # Example: CIFAR-10 has 10 classes

model = Sequential()

for i, (filters, drop_rate) in enumerate(zip([32, 64, 128], [0.2, 0.3, 0.4])):
    if i == 0:
        model.add(Conv2D(filters, (3, 3), padding='same',
                         kernel_regularizer=regularizers.l2(weight_decay),
                         input_shape=x_train.shape[1:]))  # e.g. (32, 32, 3) = x_train.shape[1:]
    else:
        model.add(Conv2D(filters, (3, 3), padding='same',
                         kernel_regularizer=regularizers.l2(weight_decay)))

    # Normalization + Activation

    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Conv2D(filters, (3, 3), padding='same',
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Activation('elu'))


    model.add(MaxPooling2D(pool_size=(2, 2)))  # Reduce spatial dimensions
    model.add(Dropout(drop_rate))              # Regularize by randomly dropping neurons


model.add(Flatten())  # Flatten 3D feature maps → 1D vector
model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification


model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 8, 8, 128)      │             

 Total params: 309,290 (1.18 MB)

 Trainable params: 308,394 (1.18 MB)

 Non-trainable params: 896 (3.50 KB)

In [10]:
#data augmentation
datagen = ImageDataGenerator(rotation_range=15,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=True
                            )

datagen.fit(x_train)

In [11]:
# configure the learning process:

opt_rms = tensorflow.keras.optimizers.RMSprop(learning_rate= 0.001)
model.compile(optimizer = opt_rms,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [12]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate

batch_size = 64

In [ ]:
#Train the model
history = model.fit(datagen.flow(x_train,y_train,
                                 batch_size = batch_size),
                                   steps_per_epoch  = x_train.shape[0] // batch_size,
                                   epochs = 30,
                                   verbose = 1,
                                   validation_data = (x_test,y_test),
                                   callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


781/781 ━━━━━━━━━━━━━━━━━━━━ 315s 400ms/step - accuracy: 0.3453 - loss: 2.1159 - val_accuracy: 0.5822 - val_loss: 1.2209 - learning_rate: 0.0010
Epoch 2/30
  1/781 ━━━━━━━━━━━━━━━━━━━━ 4:53 376ms/step - accuracy: 0.5312 - loss: 1.2575

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


781/781 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.5312 - loss: 1.2575 - val_accuracy: 0.5807 - val_loss: 1.2546 - learning_rate: 0.0010
Epoch 3/30
781/781 ━━━━━━━━━━━━━━━━━━━━ 378s 413ms/step - accuracy: 0.5543 - loss: 1.3120 - val_accuracy: 0.6275 - val_loss: 1.1352 - learning_rate: 0.0010
Epoch 4/30
781/781 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.5938 - loss: 1.2443 - val_accuracy: 0.6211 - val_loss: 1.1804 - learning_rate: 0.0010
Epoch 5/30
781/781 ━━━━━━━━━━━━━━━━━━━━ 317s 406ms/step - accuracy: 0.6239 - loss: 1.1328 - val_accuracy: 0.6892 - val_loss: 0.9625 - learning_rate: 0.0010
Epoch 6/30
781/781 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.7188 - loss: 0.8826 - val_accuracy: 0.6799 - val_loss: 1.0026 - learning_rate: 0.0010
Epoch 7/30
781/781 ━━━━━━━━━━━━━━━━━━━━ 328s 420ms/step - accuracy: 0.6535 - loss: 1.0562 - val_accuracy: 0.7067 - val_loss: 0.9267 - learning_rate: 0.0010
Epoch 8/30
781/781 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.7500 - loss: 

In [ ]:
#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

# scores[1] -- accuracy
# scores[0] -- loss

In [ ]:
scores

In [ ]:
# Prediction
test_pred = pd.DataFrame(model.predict(x_test, batch_size=200))# The model outputs a matrix of predicted probabilities (shape: (num_samples, num_digits)).
test_pred

In [ ]:
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1)) #finds the index (column name) of the maximum value in each row — i.e. the predicted class.
test_pred

In [ ]:
y_test_label = pd.DataFrame(y_test)
y_test_label

In [ ]:
y_test_label = pd.DataFrame(y_test_label.idxmax(axis = 1),columns = ['test_label'])
y_test_label

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

def show_imgs(X):
    plt.figure(figsize=(3,3))
    for k in range(16):
        plt.subplot(4, 4, k+1)
        plt.imshow(X[k])
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
show_imgs(x_train[0:16]) ## showing test image for the test label

In [ ]:
labels = ['airplane','automobile','bird','cat','deer',
          'dog','frog','horse','ship','truck']
# old code
# indices = np.argmax(model.predict(x_test[:16]),1)
# print ([labels[x] for x in indices])
# model.predict(x_test[:16]) -Produces a prediction array of shape (16, 10) — probabilities for each of the 10 classes.

# np.argmax(..., 1) -Gets the index of the maximum probability along axis 1 (i.e., the predicted class for each image).

# [labels[x] for x in indices] - Converts the predicted indices to human-readable class names.


# new one - function to show predicted image
def show_preds(X, model):
    preds = np.argmax(model.predict(X[:16]), axis=1)

    plt.figure(figsize=(3,3))
    for k in range(16):
        plt.subplot(4, 4, k+1)
        plt.imshow(X[k])
        plt.axis('off')
        plt.title(labels[preds[k]], fontsize=9)
    plt.tight_layout()
    plt.show()

In [ ]:
show_preds(x_test[0:16], model) # Actually predictions are wrong as model not fitted fully